In [3]:
import numpy as np
import pandas 
from elasticsearch import Elasticsearch
import json

In [4]:
elastic_client = Elasticsearch()


In [6]:
response = elastic_client.search( index='dossier1', body={} )

total_docs = response["hits"]["total"]["value"]
response = elastic_client.search(
    index='dossier1',
    body={},
    size=total_docs
)
elastic_docs = response["hits"]["hits"]
"""for key, val in response["hits"].items():
    if key == "hits":
        for num, doc in enumerate(val):
            print (num, '-->', doc, "\n")"""
fields = {}
for num, doc in enumerate(elastic_docs):
    pass
    # do stuff here!
    # get source data from document
    source_data = doc["_source"]
    # iterate source data (use iteritems() for Python 2)
    for key, val in source_data.items():
        try:
            fields[key] = np.append(fields[key], val)
        except KeyError:
            fields[key] = np.array([val])


    
elastic_df = pandas.DataFrame(fields)

print (elastic_df)

     nombre_de_signature id_procedure                   journee  \
0                      1          cte  2018-05-14T22:00:00.000Z   
1                      1     empotage  2018-07-20T22:00:00.000Z   
2                      0        expec  2018-08-29T22:00:00.000Z   
3                      1          cte  2018-09-21T22:00:00.000Z   
4                      0         bacm  2018-10-03T22:00:00.000Z   
5                      0         bacm  2018-10-06T22:00:00.000Z   
6                      0         bacm  2018-10-08T22:00:00.000Z   
7                      1          cte  2018-10-15T22:00:00.000Z   
8                      0        expec  2018-09-02T22:00:00.000Z   
9                      1          cte  2018-05-29T22:00:00.000Z   
10                     0        expec  2018-08-29T22:00:00.000Z   
11                     0        expec  2018-09-19T22:00:00.000Z   
12                     1          cte  2018-09-22T22:00:00.000Z   
13                     0      payment  2018-10-06T22:00:00.000

# Nettoyage des données 


In [7]:
import tensorflow as tf
tf.random.set_seed(123)
from IPython.display import clear_output
from matplotlib import pyplot as plt
import cx_Oracle


extraction du vocabulaire

In [24]:
dftrain=elastic_df
import cx_Oracle

#connection = cx_Oracle.connect("orchestra", "orchestra", "192.168.56.3/orcl")


class OraExec(object):
    _db_connection = None
    _db_cur = None

    def __init__(self):
        self._db_connection = cx_Oracle.connect('orchestra/orchestra@192.168.56.3:1521/GUCE')
        self._db_cur = self._db_connection.cursor()
    #con=OraExec()
    def getProcedures(self):
        #ver = self._db_connection.version.split(".")
        #print (ver)
        self._db_cur.execute("select ID from orchestra_process ")

       
        return self._db_cur

    
con=OraExec()
cur=con.getProcedures()
voc_procedures=[]
for result in cur:
    voc_procedures.append(result[0])
#len(voc_procedures)
    

DatabaseError: ORA-12170: TNS:Connect timeout occurred

ici on fournit les données de facons conforme: 
    on doe tout les features , et on donnes les bons types à nos column pour qu'elle puisse adhérer avec l'algorithme de onehotencoding que nous fournis tensorflow

In [86]:
dftrain=elastic_df

dftrain["journee"] = pandas.to_datetime(dftrain["journee"])
dftrain["good_hs_code"] = dftrain["good_hs_code"].astype(str) 

#dftrain.dtypes

dftrain["day"] = dftrain["journee"].dt.day
dftrain["month"] = dftrain["journee"].dt.month
dftrain["year"] = dftrain["journee"].dt.year
dftrain["weekday"] = dftrain["journee"].dt.weekday
dftrain=dftrain.drop(["@version","@timestamp","journee","type"],axis=1)

dftrain.fillna(value=pandas.np.nan, inplace=True)
y_train=dftrain['duree_moyenne']

dftrain=dftrain[['nombre_de_signature', 'dossier_en_cours',
                'nombre_de_ci','poid_moyen','qte_moyenne','nombre_de_dossier'
                  ,'nombre_de_rejection','day','month',
                'year','weekday','id_procedure', 'nom_procedure' ,'good_hs_code']]
dftrain.head()


,nombre_de_signature,dossier_en_cours,nombre_de_ci,poid_moyen,qte_moyenne,nombre_de_dossier,nombre_de_rejection,day,month,year,weekday,id_procedure,nom_procedure,good_hs_code
0,1,3,0,NaN,5.0,4,0,14,5,2018,0,cte,Certificat contrôle technique export,44034915000
1,1,3,1,NaN,100.0,4,0,20,7,2018,4,empotage,Certificat d'empotage,18010012000
2,0,3,0,NaN,1800.0,3,0,29,8,2018,2,expec,E-FORCE : Circuit de prise en charge à l'EXPORT,85366930903
3,1,0,0,NaN,100.0,1,0,21,9,2018,4,cte,Certificat contrôle technique export,18010012000
4,0,1,0,NaN,NaN,1,0,3,10,2018,2,bacm,Paiement de la taxe de vérification SGS,nan


In [89]:
CATEGORICAL_COLUMNS = ['id_procedure', 'nom_procedure' ,'good_hs_code']

                       #journee
NUMERIC_COLUMNS = ['nombre_de_signature', 'dossier_en_cours','nombre_de_ci','poid_moyen','qte_moyenne','nombre_de_dossier'
                  ,'nombre_de_rejection','day','month','year','weekday']

def one_hot_cat_column(feature_name, vocab):
    return tf.feature_column.indicator_column(
      tf.feature_column.categorical_column_with_vocabulary_list(feature_name,
                                                 vocab))
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
    # Need to one-hot encode categorical features.
    vocabulary = dftrain[feature_name].unique()
    feature_columns.append(one_hot_cat_column(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))
example = dict(dftrain)
voc_procedures = dftrain["id_procedure"].unique()

class_fc = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(
    'id_procedure',voc_procedures))
#print(class_fc)

#print('Feature value: "{}"'.format(example['id_procedure'].iloc[0]))
#print('One-hot encoded: ', tf.keras.layers.DenseFeatures([class_fc])(example).numpy())
tf.keras.layers.DenseFeatures(feature_columns)(example).numpy()

array([[1.400e+01, 3.000e+00, 1.000e+00, ..., 5.000e+00, 0.000e+00,
        2.018e+03],
       [2.000e+01, 3.000e+00, 0.000e+00, ..., 1.000e+02, 4.000e+00,
        2.018e+03],
       [2.900e+01, 3.000e+00, 0.000e+00, ..., 1.800e+03, 2.000e+00,
        2.018e+03],
       ...,
       [2.000e+00, 1.000e+00, 0.000e+00, ..., 1.200e+03, 0.000e+00,
        2.020e+03],
       [1.100e+01, 3.000e+00, 0.000e+00, ..., 1.000e+01, 2.000e+00,
        2.020e+03],
       [7.000e+00, 0.000e+00, 1.000e+00, ..., 5.000e+00, 0.000e+00,
        2.018e+03]], dtype=float32)

In [102]:
#normalisation des données en batch 
NUM_EXAMPLES = int(len(y_train)/2)

def make_input_fn(X, y, n_epochs=None, shuffle=True):
    def input_fn():
        dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
        if shuffle:
          dataset = dataset.shuffle(NUM_EXAMPLES)
        # For training, cycle thru dataset as many times as need (n_epochs=None).
        dataset = dataset.repeat(n_epochs)
        # In memory training doesn't use batching.
        dataset = dataset.batch(NUM_EXAMPLES)
        return dataset
    return input_fn

# Training and evaluation input functions.
train_input_fn = make_input_fn(dftrain, y_train)
#eval_input_fn = make_input_fn(dfeval, y_eval, shuffle=False, n_epochs=1)

In [103]:
dataset=train_input_fn()
dataset

<BatchDataset shapes: ({nombre_de_signature: (None,), dossier_en_cours: (None,), nombre_de_ci: (None,), poid_moyen: (None,), qte_moyenne: (None,), nombre_de_dossier: (None,), nombre_de_rejection: (None,), day: (None,), month: (None,), year: (None,), weekday: (None,), id_procedure: (None,), nom_procedure: (None,), good_hs_code: (None,)}, (None,)), types: ({nombre_de_signature: tf.int32, dossier_en_cours: tf.int32, nombre_de_ci: tf.int32, poid_moyen: tf.float64, qte_moyenne: tf.float64, nombre_de_dossier: tf.int32, nombre_de_rejection: tf.int32, day: tf.int64, month: tf.int64, year: tf.int64, weekday: tf.int64, id_procedure: tf.string, nom_procedure: tf.string, good_hs_code: tf.string}, tf.float64)>

In [104]:
linear_est = tf.estimator.LinearClassifier(feature_columns)

# Train model.
linear_est.train(train_input_fn, max_steps=100)

# Evaluation.
result = linear_est.evaluate(eval_input_fn)
clear_output()
print(pd.Series(result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\sophie\\AppData\\Local\\Temp\\tmpb3_bpbvi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


InvalidArgumentError: assertion failed: [Labels must be <= n_classes - 1] [Condition x <= y did not hold element-wise:] [x (head/losses/Cast:0) = ] [[4866.5][125742][514838848]...] [y (head/losses/check_label_range/Const:0) = ] [1]
	 [[{{node Assert}}]]